### 模型的存储与加载

In [19]:
import tensorflow as tf
import input_data
import os

In [3]:
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)
trX,trY,teX,teY = mnist.train.images,mnist.train.labels,mnist.test.images,mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [29]:
X = tf.placeholder('float',[None,784])
y = tf.placeholder('float',[None,10])

In [30]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape,stddev=0.01))
w_h = init_weights([784,625])
w_o = init_weights([625,10])

In [31]:
def model(X,w_h,w_o,p_keep_input,p_keep_hidden):
    X = tf.nn.dropout(X,p_keep_input)
    h = tf.nn.relu(tf.matmul(X,w_h))
    h = tf.nn.dropout(h,p_keep_hidden)
    return tf.matmul(h,w_o)

In [32]:
p_keep_input = tf.placeholder('float')
p_keep_hidden= tf.placeholder('float')

In [33]:
py_x=model(X,w_h,w_o,p_keep_input,p_keep_hidden)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x,labels=y))
train_op = tf.train.GradientDescentOptimizer(0.0001).minimize(cost)
predict_op = tf.argmax(py_x,1)

In [34]:
ckpt_dir = './ckpt_dir'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

In [35]:
saver = tf.train.Saver()#位于saver之后的变量将不会被存储
#只要提供一个计数器，当计数器触发时，Saver类可以自动的生成checkpoint文件

In [39]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(100):
        batch = mnist.train.next_batch(50)
        if i%10 == 0:
            saver.save(sess,ckpt_dir+'/model.ckpt',global_step=i) 
        sess.run(train_op,feed_dict={X:batch[0],y:batch[1],p_keep_input:0.8,p_keep_hidden:0.5})

In [42]:
#加载模型
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    ckpt = tf.train.get_checkpoint_state(ckpt_dir)
    if ckpt and ckpt.model_checkpoint_path:
        print(ckpt.model_checkpoint_path)
        print(ckpt)
        saver.restore(sess,ckpt.model_checkpoint_path)

./ckpt_dir/model.ckpt-90
model_checkpoint_path: "./ckpt_dir/model.ckpt-90"
all_model_checkpoint_paths: "./ckpt_dir/model.ckpt-50"
all_model_checkpoint_paths: "./ckpt_dir/model.ckpt-60"
all_model_checkpoint_paths: "./ckpt_dir/model.ckpt-70"
all_model_checkpoint_paths: "./ckpt_dir/model.ckpt-80"
all_model_checkpoint_paths: "./ckpt_dir/model.ckpt-90"

INFO:tensorflow:Restoring parameters from ./ckpt_dir/model.ckpt-90


### 图的存储与加载

In [43]:
v =tf.Variable(0,name='my_variable')
sess = tf.Session()
tf.train.write_graph(sess.graph_def,'./tmp/tfmodel','train.pbtxt')

'./tmp/tfmodel/train.pbtxt'

In [44]:
with tf.Session() as sess:
    with tf.gfile.FastGFile('./tmp/tfmodel/train.pbtxt','rb'):
        graph_def = tf.Graph()
        graph_def.ParseFromString(f.read())
        sess.graph.as_default()
        tf.import_graph_def(graph_def,name='tfgraph')

AttributeError: 'Graph' object has no attribute 'ParseFromString'